# Time Complexity and Approximation Analysis

In [1]:
from utils import *
from tqdm import tqdm
from itertools import product

We will analyze $G(n, \, p)$ graphs with different parameters and different terminal nodes number:

In [2]:
N = [5, 10, 15, 20, 30]
EDGE_P = [0.1, 0.3, 0.7, 0.9]
TERMINAL_P = [0.1, 0.3]
WEIGHTS_RANGE = (1, 10)

NUMBER_OF_ITERATIONS = 100  # for each n, edge_p, terminal_p

In [3]:
def compute_both_algorithms(n: int, edge_p: float, terminal_p: float, weights_range: tuple[int, int], number_of_iterations: int):
    Gnp_graphs = generate_Gnp_weighted_graphs(n, edge_p, number_of_iterations, weights_range)
    terminals_boolean_array = generate_terminal_boolean_array(number_of_iterations, n, terminal_p)

    solutions = []

    for i in range(number_of_iterations):
        edges = Gnp_graphs[i]
        terminals = list(np.where(terminals_boolean_array[i])[0] + 1)

        dreyfus_wagner_solution = compute_dreyfus_wagner(n, edges, terminals)
        approximate_solution = compute_2_approximation(n, edges, terminals)

        solutions.append((
            (dreyfus_wagner_solution[0][0], dreyfus_wagner_solution[0][1], dreyfus_wagner_solution[1][1] - dreyfus_wagner_solution[1][0]),
            (approximate_solution[0][0], approximate_solution[0][1], approximate_solution[1][1] - approximate_solution[1][0])
        ))

    return solutions

In [4]:
gnp_results = {}

results = []
for n, edge_p, terminal_p in tqdm(product(N, EDGE_P, TERMINAL_P), total=len(N) * len(EDGE_P) * len(TERMINAL_P)):
    result = compute_both_algorithms(n, edge_p, terminal_p, WEIGHTS_RANGE, NUMBER_OF_ITERATIONS)
    gnp_results[(n, edge_p, terminal_p)] = result

100%|██████████| 40/40 [03:00<00:00,  4.52s/it]


Let's save our results:

In [4]:
import os
import json
from pathlib import Path


notebook_path = os.path.abspath("analysis.ipynb")
BASE_DIR = Path(notebook_path).parent.parent

In [ ]:
gnp_results_serializable = {str(key): value for key, value in gnp_results.items()}

with open(str(BASE_DIR / "data/Gnp_results.json"), 'w') as file:
    file.write(json.dumps(gnp_results_serializable))

In [5]:
with open(str(BASE_DIR / "data/Gnp_results.json"), 'r') as file:
    gnp_results_serializable = json.loads(file.read().strip())
    gnp_results = {eval(key): value for key, value in gnp_results_serializable.items()}

Helper function to assess:

In [26]:
import pandas as pd


def get_statistics(n: int, edge_p: float, terminal_p: float):
    # filtering for results where there is some feasible solution
    data = list(filter(lambda x: x[1][0], gnp_results[(n, edge_p, terminal_p)]))
    length = len(data)

    # separating
    dreyfus_wagner = list(map(lambda x: x[0], data))
    approximate = list(map(lambda x: x[1], data))

    # calculating coefficients
    approximation_coefficients = list(map(
        lambda x: 1.0 if x[1][1] == 0 else x[1][1] / x[0][1],
        zip(dreyfus_wagner, approximate)
    ))

    return length, {
        "dreyfus_wagner": {
            "time_complexity": {
                "average": sum(map(lambda x: x[2], dreyfus_wagner)) / length,
                "max": max(dreyfus_wagner, key=lambda x: x[2])[2]
            },
            "approximation_coefficient": {
                "average": 1.0,
                "max": 1.0
            }
        },
        "2_approximation": {
            "time_complexity": {
                "average": sum(map(lambda x: x[2], approximate)) / length,
                "max": max(approximate, key=lambda x: x[2])[2]
            },
            "approximation_coefficient": {
                "average": sum(approximation_coefficients) / length,
                "max": max(approximation_coefficients)
            }
        }
    }


def generate_tables(terminal_p: float, statistic="max"):
    return {
        "dreyfus_wagner": {
            "time_complexity": pd.DataFrame({
                f"n={n}": [get_statistics(n, p, terminal_p)[1]["dreyfus_wagner"]["time_complexity"][statistic] for p in EDGE_P]
                for n in N
            }, index=[f"p={p}" for p in EDGE_P]),
        },
        "2_approximation": {
            "time_complexity": pd.DataFrame({
                f"n={n}": [get_statistics(n, p, terminal_p)[1]["2_approximation"]["time_complexity"][statistic] for p in EDGE_P]
                for n in N
            }, index=[f"p={p}" for p in EDGE_P]),
            "approximation_coefficient": pd.DataFrame({
                f"n={n}": [round(get_statistics(n, p, terminal_p)[1]["2_approximation"]["approximation_coefficient"][statistic], 3) for p in EDGE_P]
                for n in N
            }, index=[f"p={p}" for p in EDGE_P]),
        }
    }

Tables for approximation coefficient:

In [31]:
generate_tables(0.1, statistic="max")["2_approximation"]["approximation_coefficient"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,1.0,1.0,1.0,1.0,1.219
p=0.3,1.0,1.0,1.0,1.0,1.250
p=0.7,1.0,1.0,1.0,1.0,1.333
p=0.9,1.0,1.0,1.0,1.0,1.250


In [32]:
generate_tables(0.3, statistic="max")["2_approximation"]["approximation_coefficient"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,1.0,1.000,1.200,1.235,1.229
p=0.3,1.0,1.231,1.231,1.267,1.179
p=0.7,1.0,1.250,1.250,1.222,1.273
p=0.9,1.0,1.167,1.286,1.200,1.250


In [33]:
generate_tables(0.1, statistic="average")["2_approximation"]["approximation_coefficient"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,1.0,1.0,1.0,1.0,1.034
p=0.3,1.0,1.0,1.0,1.0,1.042
p=0.7,1.0,1.0,1.0,1.0,1.042
p=0.9,1.0,1.0,1.0,1.0,1.051


In [34]:
generate_tables(0.3, statistic="average")["2_approximation"]["approximation_coefficient"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,1.0,1.000,1.045,1.053,1.050
p=0.3,1.0,1.015,1.040,1.047,1.055
p=0.7,1.0,1.027,1.038,1.043,1.058
p=0.9,1.0,1.017,1.050,1.042,1.056


Tables for time complexity:

In [36]:
generate_tables(0.1, statistic="average")["dreyfus_wagner"]["time_complexity"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,0.01,0.05,0.01,0.05,0.857143
p=0.3,0.02,0.00,0.01,0.17,0.550000
p=0.7,0.01,0.01,0.01,0.17,0.620000
p=0.9,0.02,0.06,0.02,0.14,0.550000


In [37]:
generate_tables(0.1, statistic="average")["2_approximation"]["time_complexity"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,0.01,0.03,0.02,0.016667,0.071429
p=0.3,0.03,0.03,0.01,0.020000,0.040000
p=0.7,0.04,0.00,0.01,0.110000,0.030000
p=0.9,0.00,0.03,0.01,0.050000,0.040000


In [38]:
generate_tables(0.3, statistic="average")["dreyfus_wagner"]["time_complexity"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,0.02,0.00000,0.476190,6.135135,245.79661
p=0.3,0.01,0.05814,0.927835,7.280000,252.00000
p=0.7,0.00,0.07000,0.880000,7.360000,252.07000
p=0.9,0.02,0.04000,0.890000,7.370000,249.73000


In [39]:
generate_tables(0.3, statistic="average")["2_approximation"]["time_complexity"]

,n=5,n=10,n=15,n=20,n=30
p=0.1,0.00,0.000000,0.000000,0.054054,0.067797
p=0.3,0.02,0.011628,0.010309,0.060000,0.110000
p=0.7,0.01,0.010000,0.060000,0.060000,0.050000
p=0.9,0.02,0.050000,0.020000,0.050000,0.060000
